In [2]:
import os

from ladybug.sql import SQLiteResult
from ladybug import analysisperiod as ap

from eppy import *
from eppy.modeleditor import IDF

import pickle

In [3]:
root = os.getcwd()
# get group model to be modified

model_root = "/Users/julietnwagwuume-ezeoke/My Drive/CEE256_BuildSys/final_256/Data/Calibration/group_models/"
# Final_project_3-6-2022_EH6, Final_project_3-3-2022_EC1, Final_project_3-2-2022_1, Final_project_2-25-2022_3-1
model_name = "Final_project_3-10-2022_EH8" 

model_dir = os.path.join(model_root, model_name)
# model_dir = os.path.join(root, "group_models/Final_project_3-10-2022_EH6_sac_ddy/run")

In [4]:
# set the idd
iddfile = "/Applications/OpenStudioApplication-1.1.1/EnergyPlus/Energy+.idd"
IDF.setiddname(iddfile)

# get the idf 
idfPath = os.path.join(model_dir, "in.idf")
epw = "/Users/julietnwagwuume-ezeoke/Documents/cee256_local/weather_files/CA_PALO-ALTO-AP_724937S_19.epw"

# # make a new dir for the output
adjusted_model_dir = os.path.join(root, "eppy_adjusted_models_2")
## NEED TO CHANGE THE LAST STRING TO CREATE NEW FOLDER!!!


### Init New IDF + Making the Files Readable by SQL

In [5]:
def prep_idf_change():
    idf0 = IDF(idfPath, epw)

    intVarsPath = os.path.join(root, "intVars")
    with open(intVarsPath, "rb") as f:
        intVars = pickle.load(f)
    intVars

    # update the outputs with interesting variables 
    currVars = [var.Variable_Name for var in idf0.idfobjects["Output:Variable"]]
    for var in intVars:
        if var not in currVars:
            newObj = idf0.newidfobject("Output:Variable", Key_Value='', Variable_Name=var)

    # change all reporting frequency
    allVars = [var for var in idf0.idfobjects["Output:Variable"]]
    for var in allVars:
        var.Reporting_Frequency = "Hourly"

    allVars = [var for var in idf0.idfobjects["Output:Meter"]]
    for var in allVars:
        var.Reporting_Frequency = "Hourly"

    # change the run period
    runPeriod = idf0.idfobjects["RunPeriod"][0]
    runPeriod.Begin_Month = 1 
    runPeriod.Begin_Day_of_Month = 1
    runPeriod.End_Month = 12
    runPeriod.End_Day_of_Month = 31


    # remove extraneous electricity:facility terms 
    # for some reason openstudio adds three of them which confuses
    # the program that reads sql
    extra_elect = [var for var in idf0.idfobjects["Output:Meter:MeterFileOnly"]]

    for e in extra_elect:
        idf0.removeidfobject(e)
    check = [var for var in idf0.idfobjects["Output:Meter:MeterFileOnly"]]
     # should see a blank array showing these were removed
    print(check)

    return idf0



In [73]:
# new summer elect shed

def summer_elect():
    # new idf daily schedule eobject 
    dayObj = idf0.newidfobject("Schedule:Day:Interval", 
    Name="Medium Office Bldg Equip Summertime Schedule",
    Schedule_Type_Limits_Name = "Fraction",
    Time_1 = "09:00",
    Value_Until_Time_1 = 0.1,
    Time_2 = "17:00",
    Value_Until_Time_2 = 0.8,
    Time_3 = "24:00",
    Value_Until_Time_3 = 0.1
    )

    # new idf weekly schedule object 
    sched_to_mod = [s  for s in idf0.idfobjects["Schedule:Week:Daily"]][0]
    weekObj = idf0.copyidfobject(sched_to_mod)
    weekObj.Name = "Medium Office Bldg Equip Week Rule - May-August"

    # edit schedule weeks
    sched_week_to_mod = [s  for s in idf0.idfobjects["Schedule:Week:Daily"] if s.Name == weekObj.Name][0]
    for field in sched_week_to_mod.fieldnames:
        if "ScheduleDay_Name" in field:
            sched_week_to_mod[field] ="Medium Office Bldg Equip Summertime Schedule"

    # new summer time sched
    sched_year_to_mod = [s  for s in idf0.idfobjects["Schedule:Year"] if s.Name == "Medium Office Bldg Equip"][0]
    sched_year_to_mod.End_Month_2 = 5
    sched_year_to_mod.End_Day_2 = 25
    # switch summer sched
    sched_year_to_mod.ScheduleWeek_Name_3 = "Medium Office Bldg Equip Week Rule - May-August"
    sched_year_to_mod.Start_Month_3 = 5
    sched_year_to_mod.Start_Day_3 = 26
    sched_year_to_mod.End_Month_3 = 9
    sched_year_to_mod.End_Day_3 = 15
    # back to norm sched 
    sched_year_to_mod.ScheduleWeek_Name_4 = "Medium Office Bldg Equip Week Rule - Jan6-Dec31"
    sched_year_to_mod.Start_Month_4 = 9
    sched_year_to_mod.Start_Day_4 = 16
    sched_year_to_mod.End_Month_4 = 12
    sched_year_to_mod.End_Day_4 = 31

### Calibration

In [154]:
idf0 = prep_idf_change()

[]


In [155]:
adjust_string_name = "Final_project_3_11_2022_JNU"

infil_changes = [s  for s in idf0.idfobjects["ZoneInfiltration:DesignFlowRate"]]
for i in infil_changes:
    i.Flow_per_Exterior_Surface_Area = 0.0017 # prev 0.000226568, 

In [156]:
# sizing params 
sz_params = [s  for s in idf0.idfobjects["Sizing:Parameters"]][0]
sz_params.Cooling_Sizing_Factor = 0.6

In [157]:
chill_water_changes = [s  for s in idf0.idfobjects["Schedule:Day:Interval"] if s.Name == "Chilled_Water_Temperature_Default"][0]
chill_water_changes.Value_Until_Time_1 = 12

In [158]:
cool_set_changes = [s  for s in idf0.idfobjects["Schedule:Day:Interval"] if s.Name == "Medium Office ClgSetp Default Schedule"][0]
cool_set_changes.Value_Until_Time_2 = 25 #c , previous 24

In [159]:
# adjust summer schedule
sched_to_mod = [s  for s in idf0.idfobjects["Schedule:Day:Interval"] if s.Name == "Medium Office Bldg Occ Rule 2 Day Schedule"][0]
sched_to_mod.Value_Until_Time_3 = 0.1

In [160]:
# adjust shgc
window_mat = [s  for s in idf0.idfobjects["WindowMaterial:SimpleGlazingSystem"] if s.Name == "Thornton Window Material 1"][0]
window_mat.Solar_Heat_Gain_Coefficient = 0.25 # prev 0.31
window_mat.UFactor = 1.20
# windows.Value_Until_Time_3 = 0.1

In [161]:
# increase elect 
elect = [s  for s in idf0.idfobjects["ElectricEquipment"] if s.Name != "Electric Equipment 6"]
for e in elect:
    e.Watts_per_Zone_Floor_Area = 20 # prev 6

In [162]:
# edit default bulding schedule
def_equip = [s  for s in idf0.idfobjects["Schedule:Day:Interval"] if s.Name == "Medium Office Bldg Equip Default Schedule"][0]
def_equip.Time_1 = "5:00"
def_equip.Value_Until_Time_1 = 0.7

In [163]:
# even less people in summer 
summer_occ = [s  for s in idf0.idfobjects["Schedule:Day:Interval"] if s.Name == "Medium Office Bldg Occ Rule 2 Day Schedule"][0]
summer_occ.Time_2 = "10:00"
summer_occ.Time_3 = "16:00"
summer_occ.Value_Until_Time_2 = 0
summer_occ.Value_Until_Time_3 = 0.05

In [164]:
# summer electricity schedule from above 
summer_elect()

### Run

In [165]:
# create new directory to save the file 
new_dir_name = os.path.join(adjusted_model_dir, adjust_string_name)
os.makedirs(new_dir_name)

# save the updated idf there
idf0.save(os.path.join(new_dir_name, "in2.idf"))

# run the idf 
idf0.run(output_directory=new_dir_name)


/Applications/OpenStudioApplication-1.1.1/EnergyPlus/energyplus --weather /Users/julietnwagwuume-ezeoke/Documents/cee256_local/weather_files/CA_PALO-ALTO-AP_724937S_19.epw --output-directory /Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/eppy_adjusted_models_2/Final_project_3_11_2022_JNU --idd /Applications/OpenStudioApplication-1.1.1/EnergyPlus/Energy+.idd /Users/julietnwagwuume-ezeoke/Documents/cee256_local/_final_project_local/in.idf

EnergyPlus Starting
EnergyPlus, Version 9.4.0-998c4b761e, YMD=2022.03.11 04:00


Could not find platform independent libraries <prefix>
Could not find platform dependent libraries <exec_prefix>
Consider setting $PYTHONHOME to <prefix>[:<exec_prefix>]


Initializing Response Factors
Calculating CTFs for "ASHRAE 90.1-2010 EXTROOF IEAD CLIMATEZONE 2-8"
Calculating CTFs for "EXTSLABCARPET 4IN CLIMATEZONE 1-8"
Calculating CTFs for "INTERIOR CEILING"
Calculating CTFs for "INTERIOR WALL"
Calculating CTFs for "THORNTON EXT WALL"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Initializing Solar Heat Gains
Initializing Internal Heat Gains
Initializing Interior Solar 

EnergyPlus Completed Successfully.


In [166]:
# vav_changes = [s  for s in idf0.idfobjects["AirTerminal:SingleDuct:VAV:Reheat"]]
# for vav in vav_changes:
#     # vav.Maximum_Hot_Water_or_Steam_Flow_Rate = 0.05
#     vav.Maximum_Air_Flow_Rate = 1.5
#     # vav.Fixed_Minimum_Air_Flow_Rate = 0.7 ignored
#     # vav.Maximum_Flow_per_Zone_Floor_Area_During_Reheat = 0.9 ignored
#     vav.Maximum_Flow_Fraction_During_Reheat = 0.3
#     vav.Constant_Minimum_Air_Flow_Fraction = 0.3